In [ ]:
import requests
API_KEY = "tHCDG5tygZ34vwbIQr7g"
BASE_URL = "https://api.opencorporates.com/v0.4/companies/search"


In [26]:
params = {
    "api_token": API_KEY,
    "q": "pipe fabrication",
    "jurisdiction_code": "us",
    "industry_codes": "us_naics-332996",  # Metal pipe/tube manufacturing
    "current_status": "Active",
    "fields": "name,company_number,registered_address",
    "per_page": 50,
    "page": 1
}


In [27]:
def get_pipe_fabricators():
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        results = response.json()['results']['companies']
        return [{
            'legal_name': c['company']['name'],
            'jurisdiction': c['company']['jurisdiction_code'],
            'company_number': c['company']['company_number'],
            'opencorporates_url': c['company']['opencorporates_url'],
            'registered_address': c['company']['registered_address_in_full']
        } for c in results if 'pipe' in c['company']['name'].lower()]
    else:
        print(f"API Error: {response.status_code}")
        return []


In [28]:
def enrich_with_procurement_data(oc_data):
    # Add procurement-specific fields
    for vendor in oc_data:
        vendor.update({
            'materials_supported': "304L SS, 9% Ni" if "cryogenic" in vendor['legal_name'].lower() else "304L SS",
            'certifications': "ASME B31.3",
            'otif_12mo': round(np.random.triangular(85, 92, 99), 1),
            'capacity_units': int(np.random.normal(5000, 1500))
        })
    return oc_data


In [30]:
# Get real vendors from OpenCorporates
real_vendors = enrich_with_procurement_data(get_pipe_fabricators())

# Generate synthetic vendors
synthetic_vendors = generate_synthetic_vendors(50 - len(real_vendors))  # From previous implementation

# Combine datasets
supplier_master = pd.DataFrame(real_vendors + synthetic_vendors)
supplier_master.to_csv("supplier_master_v2.csv", index=False)


API Error: 403


In [31]:
import requests
import pandas as pd
import numpy as np

API_KEY  = "tHCDG5tygZ34vwbIQr7g"
BASE_URL = "https://api.opencorporates.com/v0.4/companies/search.json"

def get_pipe_fabricators_from_oc(per_page=50, page=1):
    params = {
        "api_token":         API_KEY,
        "q":                 "pipe fabrication",
        "jurisdiction_code": "us",
        "industry_codes":    "us_naics-332996",
        "current_status":    "Active",
        "per_page":          per_page,
        "page":              page
    }
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    data = r.json()["results"]["companies"]
    vendors = []
    for c in data:
        name = c["company"]["name"]
        if "pipe" not in name.lower():
            continue
        vendors.append({
            "legal_name":            name,
            "jurisdiction":          c["company"]["jurisdiction_code"],
            "company_number":        c["company"]["company_number"],
            "opencorporates_url":    c["company"]["opencorporates_url"],
            "registered_address":    c["company"]["registered_address_in_full"]
        })
    return vendors

def enrich_with_procurement_data(vendors):
    for v in vendors:
        v.update({
            "materials_supported": "304L SS, 9% Ni Alloy",
            "certifications":      "ASME B31.3",
            "otif_12mo":           round(np.random.triangular(85, 92, 99), 1),
            "capacity_units":      int(max(1000, np.random.normal(5000, 1500)))
        })
    return vendors

if __name__ == "__main__":
    real_vendors = get_pipe_fabricators_from_oc()
    real_vendors = enrich_with_procurement_data(real_vendors)
    df_real = pd.DataFrame(real_vendors)
    # If you need N=50 total, generate synthetic for the remainder:
    N_TARGET = 50
    n_synth = max(0, N_TARGET - len(df_real))
    # Assume you have a function generate_synthetic_vendors(n)
    from your_synthesis_module import generate_synthetic_vendors
    synthetic = generate_synthetic_vendors(n_synth)
    df = pd.concat([df_real, pd.DataFrame(synthetic)], ignore_index=True)
    df.to_csv("data/Bayou_Freeze/supplier_master_v2.csv", index=False)
    print(f"Seeded {len(df_real)} real vendors, added {n_synth} synthetic → supplier_master_v2.csv")


HTTPError: 404 Client Error: Not Found for url: https://api.opencorporates.com/v0.4/companies/search.json?api_token=tHCDG5tygZ34vwbIQr7g&q=pipe+fabrication&jurisdiction_code=us&industry_codes=us_naics-332996&current_status=Active&per_page=50&page=1

In [33]:
import requests
import pandas as pd
import numpy as np

API_KEY  = "tHCDG5tygZ34vwbIQr7g"
BASE_URL = "https://api.opencorporates.com/v0.4/companies/search"

def get_pipe_fabricators_from_oc(per_page=50, page=1):
    params = {
        "api_token":         API_KEY,
        "q":                 "pipe fabrication",
        "jurisdiction_code": "us",
        "industry_codes":    "us_naics-332996",
        "current_status":    "Active",
        "per_page":          per_page,
        "page":              page
    }
    headers = {"Accept": "application/json"}
    r = requests.get(BASE_URL, params=params, headers=headers)
    r.raise_for_status()
    data = r.json()["results"]["companies"]
    vendors = []
    for c in data:
        name = c["company"]["name"]
        if "pipe" not in name.lower():  # simple filter
            continue
        vendors.append({
            "legal_name":            name,
            "jurisdiction":          c["company"]["jurisdiction_code"],
            "company_number":        c["company"]["company_number"],
            "opencorporates_url":    c["company"]["opencorporates_url"],
            "registered_address":    c["company"]["registered_address_in_full"]
        })
    return vendors

def enrich_with_procurement_data(vendors):
    for v in vendors:
        v.update({
            "materials_supported": "304L SS, 9% Ni Alloy",
            "certifications":      "ASME B31.3",
            "otif_12mo":           round(np.random.triangular(85, 92, 99), 1),
            "capacity_units":      int(max(1000, np.random.normal(5000, 1500)))
        })
    return vendors

if __name__ == "__main__":
    real_vendors = get_pipe_fabricators_from_oc()
    real_vendors = enrich_with_procurement_data(real_vendors)
    df_real = pd.DataFrame(real_vendors)


# Supplier Data Sourcing: Attempts & Lessons Learned

## Project Context  
We are simulating Bechtel’s procurement workflow for the “Bayou Freeze” cryogenic-piping project. A core requirement is a credible **supplier master** to power our GenAI-driven supplier-recommendation logic. Over the course of our prototyping, we explored several public APIs and scraping approaches—each with unique challenges.

---

## 1. SAM.gov Entity API

- **Goal**: Retrieve active pipe-fabrication vendors by PSC code.
- **Approach**  
  - Tried parameters `psc`, `limit`; then `relevantPscCode`, `pageSize`, `pageNum`.  
  - Attempted both v1 and v3 endpoints.
- **Outcome**  
  - **400 / 422 errors** (“invalid search parameter”) due to shifting schema.  
  - **Rate limit**: 10 requests/day for non-federal personal key.  
  - **Conclusion**: Unreliable for bulk harvesting; fencing in schema and quotas.

---

## 2. USAspending.gov Awards Search API

- **Goal**: Fetch prime contract award recipients in NAICS 332996 (Pipe & Fittings).
- **Approach**  
  - Posted to `/api/v2/search/spending_by_award/` with nested `filters` (award types, NAICS, time period) and `fields`.  
  - Tried multiple JSON payload variations to satisfy the contract.
- **Outcome**  
  - **422 Unprocessable Entity** unless payload matched exact contract schema.  
  - Returned awardees spanned IT and services firms—few true pipe fabricators.  
  - **Conclusion**: Schema-strict with mixed-relevance results; not ideal for seeding.

---

## 3. OpenCorporates API

- **Goal**: Look up “pipe fabrication” companies in U.S. by NAICS.
- **Approach**  
  - Queried `/companies/search` with `industry_codes=us_naics-332996`, `jurisdiction_code=us`.  
  - Experimented with `.json` suffix and `Accept: application/json` header.
- **Outcome**  
  - **403 / 404 errors** due to endpoint restrictions or mis-routing.  
  - Empty or inconsistent results—“pipe” not well indexed.  
  - **Conclusion**: Free-tier token present but endpoint not suited to this query.

---

## 4. HTML Scraping OpenCorporates

- **Goal**: Fall-back to scraping the public search UI for “pipe fabrication.”
- **Approach**  
  - Parsed the first 5–10 table rows via BeautifulSoup.
- **Outcome**  
  - Successful retrieval of a handful of vendor names and URLs.  
  - Requires custom parsing logic; fragile to UI changes.  
  - **Conclusion**: Viable for a small seed set, but not scalable.

---

## 5. Final “Seed + Synthetic” Strategy

1. **Manual Seed (5–10 Vendors)**  
   - Hand-select bona fide pipe-fabrication shops from public directories, LinkedIn, or a curated web search.  
   - Populate `supplier_seed.csv` with their core details (name, jurisdiction, certifications).

2. **Synthetic Expansion (40–45 Vendors)**  
   - Programmatically generate additional records with realistic attributes:  
     - `materials_supported`: “304L SS, 9 % Ni Alloy”  
     - `certifications`: “ASME B31.3, ISO 9001”  
     - Performance metrics (OTIF %, defect_rate %) sampled from industry-standard distributions.  
     - Annual capacity estimates (tons/year).

3. **Unified Supplier Master**  
   - Merge seeded and synthetic rows into `supplier_master.csv`.  
   - Use this stable, reproducible dataset for:  
     - **Supplier short-listing** (filter by spec and rank by OTIF/capacity).  
     - **RFQ generated panels** (embed top 3 suppliers per BOM line).

---

## Key Takeaways

- **APIs are fragile**: schema changes, rate limits, and data relevance issues hinder rapid prototyping in a student project.  
- **Hybrid approach** offers the best of both worlds: authentic “real” entries for credibility, synthetic scale for coverage and offline repeatability.  
- **Documentation of assumptions** (e.g., generation logic, seed source) is critical for transparency and grading.

---

*Prepared by the “Bayou Freeze” student project team, emulating Bechtel’s enterprise procurement standards.*  


In [37]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime

# === Configuration ===
TOTAL_SUPPLIERS = 50
GULF_STATES = ["us_tx", "us_la", "us_al", "us_ms"]
PAYMENT_TERMS = ["NET_30", "NET_45"]
CURRENCY_CODE = "USD"
TODAY = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
np.random.seed(42)
faker = Faker()

# === Seed Data Definition ===
seed_records = [
    {"supplier_id": 1, "company_name": "Superior Pipe Fittings, Inc.", "jurisdiction_code": "us_tx", "company_number": "12345678",
     "payment_terms": "NET_30", "currency_code": "USD", "capacity_tpy": 3500,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": True},
    {"supplier_id": 2, "company_name": "Midwest Cryogenic Fabrication LLC", "jurisdiction_code": "us_la", "company_number": "87654321",
     "payment_terms": "NET_30", "currency_code": "USD", "capacity_tpy": 4200,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": False},
    {"supplier_id": 3, "company_name": "Gulf Coast Nickel Alloys Inc.", "jurisdiction_code": "us_al", "company_number": "23456789",
     "payment_terms": "NET_45", "currency_code": "USD", "capacity_tpy": 2800,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": True},
    {"supplier_id": 4, "company_name": "CryoSteel Services LLC", "jurisdiction_code": "us_tx", "company_number": "98765432",
     "payment_terms": "NET_30", "currency_code": "USD", "capacity_tpy": 3000,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": False, "conflict_minerals": False},
    {"supplier_id": 5, "company_name": "Deep South Pipe & Tube Co.", "jurisdiction_code": "us_la", "company_number": "34567890",
     "payment_terms": "NET_30", "currency_code": "USD", "capacity_tpy": 5000,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": True},
    {"supplier_id": 6, "company_name": "Alamo Cryogenics Inc.", "jurisdiction_code": "us_tx", "company_number": "45678901",
     "payment_terms": "NET_45", "currency_code": "USD", "capacity_tpy": 2600,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": False, "conflict_minerals": True},
    {"supplier_id": 7, "company_name": "Appalachian Pipe Fabricators", "jurisdiction_code": "us_ms", "company_number": "56789012",
     "payment_terms": "NET_30", "currency_code": "USD", "capacity_tpy": 3200,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": False}
]

seed_df = pd.DataFrame(seed_records)

# === Synthetic Generation ===
synthetic_records = []
for sid in range(8, TOTAL_SUPPLIERS + 1):
    synthetic_records.append({
        "supplier_id": sid,
        "company_name": f"{faker.company()} Pipe & Tube",
        "jurisdiction_code": np.random.choice(GULF_STATES),
        "company_number": faker.unique.random_number(digits=8, fix_len=True),
        "payment_terms": np.random.choice(PAYMENT_TERMS),
        "currency_code": CURRENCY_CODE,
        "capacity_tpy": int(np.random.uniform(1000, 6000)),
        "created_at": TODAY,
        "updated_at": TODAY,
        "reach_compliant": bool(np.random.choice([True, False], p=[0.8, 0.2])),
        "conflict_minerals": bool(np.random.choice([True, False], p=[0.7, 0.3]))
    })

synthetic_df = pd.DataFrame(synthetic_records)

# === Combine & Save ===
master_df = pd.concat([seed_df, synthetic_df], ignore_index=True)
OUTPUT_PATH = 'supplier_master_regional.csv'
master_df.to_csv(OUTPUT_PATH, index=False)

print(f"Synthetic generation complete. Supplier master saved to: {OUTPUT_PATH}")

# Display sample
print("Supplier Master (Sample)", master_df.sample(5))


Synthetic generation complete. Supplier master saved to: supplier_master_regional.csv
Supplier Master (Sample)     supplier_id                 company_name jurisdiction_code company_number  \
34           35      Elliott PLC Pipe & Tube             us_ms       13719880   
21           22  Williams-Miller Pipe & Tube             us_ms       12308932   
13           14     Brown-Nguyen Pipe & Tube             us_la       55080854   
48           49        Curry Inc Pipe & Tube             us_ms       20129395   
7             8       Yang-Silva Pipe & Tube             us_al       95701021   

   payment_terms currency_code  capacity_tpy            created_at  \
34        NET_30           USD          1384  2025-05-10T11:03:22Z   
21        NET_45           USD          1979  2025-05-10T11:03:22Z   
13        NET_45           USD          4925  2025-05-10T11:03:22Z   
48        NET_45           USD          5176  2025-05-10T11:03:22Z   
7         NET_45           USD          5753  2025-0

/tmp/ipykernel_1092/243852492.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  TODAY = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


In [ ]:
# import pandas as pd
# import numpy as np

# # Configuration
# NUM_BOM_LINES = 210
# MATERIAL_CODES = ['304L_SS', '9%_Ni_Alloy']
# NPS_OPTIONS = ['1/2', '3/4', '1', '2', '3', '4', '6']
# SCHEDULE_OPTIONS = ['SCH 40', 'SCH 80']
# SPEC_CODES = ['ASME_B31.3', 'NACE_MR0175']
# TEST_TEMP_C = -196  # Required cryogenic impact test temperature

# np.random.seed(42)

# # Generate BOM
# bom_records = []
# for i in range(1, NUM_BOM_LINES + 1):
#     material = np.random.choice(MATERIAL_CODES, p=[0.6, 0.4])
#     bom_records.append({
#         'bom_line_id': i,
#         'material_code': material,
#         'nps': np.random.choice(NPS_OPTIONS, p=[0.1, 0.1, 0.2, 0.2, 0.15, 0.15, 0.1]),
#         'schedule': np.random.choice(SCHEDULE_OPTIONS, p=[0.7, 0.3]),
#         'spec_code': 'ASME_B31.3' if material == '304L_SS' else np.random.choice(SPEC_CODES),
#         'quantity': np.random.randint(1, 21),
#         'test_temp_c': TEST_TEMP_C
#     })

# bom_df = pd.DataFrame(bom_records)
# OUTPUT_PATH = 'bom.csv'
# bom_df.to_csv(OUTPUT_PATH, index=False)

# # Display sample lines
# print("BOM Sample (First 5 Rows)", bom_df.head())



BOM Sample (First 5 Rows)    bom_line_id material_code  nps schedule   spec_code  quantity  test_temp_c
0            1       304L_SS    6   SCH 80  ASME_B31.3         7         -196
1            2       304L_SS  1/2   SCH 40  ASME_B31.3         4         -196
2            3       304L_SS    3   SCH 40  ASME_B31.3        12         -196
3            4   9%_Ni_Alloy  1/2   SCH 80  ASME_B31.3        12         -196
4            5   9%_Ni_Alloy  1/2   SCH 40  ASME_B31.3        10         -196


In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import date, timedelta

# === Configuration ===
faker = Faker()
np.random.seed(42)

# Constants
TOTAL_SUPPLIERS = 50
GULF_STATES = ["us_tx", "us_la", "us_al", "us_ms"]
PAYMENT_TERMS = ["NET_30", "NET_45"]
CURRENCY_CODE = "USD"
TODAY = date(2025, 5, 10)

# === 1. Supplier Master ===
seed_suppliers = [
    {"supplier_id": 1, "company_name": "Superior Pipe Fittings, Inc.", "jurisdiction_code": "us_tx", "company_number": "12345678",
     "payment_terms": "NET_30", "currency_code": CURRENCY_CODE, "capacity_tpy": 3500,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": True},
    {"supplier_id": 2, "company_name": "Midwest Cryogenic Fabrication LLC", "jurisdiction_code": "us_la", "company_number": "87654321",
     "payment_terms": "NET_30", "currency_code": CURRENCY_CODE, "capacity_tpy": 4200,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": False},
    {"supplier_id": 3, "company_name": "Gulf Coast Nickel Alloys Inc.", "jurisdiction_code": "us_al", "company_number": "23456789",
     "payment_terms": "NET_45", "currency_code": CURRENCY_CODE, "capacity_tpy": 2800,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": True},
    {"supplier_id": 4, "company_name": "CryoSteel Services LLC", "jurisdiction_code": "us_tx", "company_number": "98765432",
     "payment_terms": "NET_30", "currency_code": CURRENCY_CODE, "capacity_tpy": 3000,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": False, "conflict_minerals": False},
    {"supplier_id": 5, "company_name": "Deep South Pipe & Tube Co.", "jurisdiction_code": "us_la", "company_number": "34567890",
     "payment_terms": "NET_30", "currency_code": CURRENCY_CODE, "capacity_tpy": 5000,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": True},
    {"supplier_id": 6, "company_name": "Alamo Cryogenics Inc.", "jurisdiction_code": "us_tx", "company_number": "45678901",
     "payment_terms": "NET_45", "currency_code": CURRENCY_CODE, "capacity_tpy": 2600,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": False, "conflict_minerals": True},
    {"supplier_id": 7, "company_name": "Appalachian Pipe Fabricators", "jurisdiction_code": "us_ms", "company_number": "56789012",
     "payment_terms": "NET_30", "currency_code": CURRENCY_CODE, "capacity_tpy": 3200,
     "created_at": TODAY, "updated_at": TODAY, "reach_compliant": True, "conflict_minerals": False}
]
sup_seed_df = pd.DataFrame(seed_suppliers)

# Synthetic suppliers
synthetic = []
for sid in range(8, TOTAL_SUPPLIERS+1):
    synthetic.append({
        "supplier_id": sid,
        "company_name": f"{faker.company()} Pipe & Tube",
        "jurisdiction_code": np.random.choice(GULF_STATES),
        "company_number": faker.unique.random_number(digits=8, fix_len=True),
        "payment_terms": np.random.choice(PAYMENT_TERMS),
        "currency_code": CURRENCY_CODE,
        "capacity_tpy": int(np.random.uniform(1000, 6000)),
        "created_at": TODAY,
        "updated_at": TODAY,
        "reach_compliant": bool(np.random.choice([True, False], p=[0.8,0.2])),
        "conflict_minerals": bool(np.random.choice([True, False], p=[0.7,0.3]))
    })
sup_synth_df = pd.DataFrame(synthetic)

supplier_master = pd.concat([sup_seed_df, sup_synth_df], ignore_index=True)

# === 2. BOM for Bayou Freeze ===
NUM_LINES = 210
MATERIAL_CODES = ['304L_SS', '9%_Ni_Alloy']
NPS_OPTIONS = ['1/2','3/4','1','2','3','4','6']
SCHEDULE_OPTIONS = ['SCH 40','SCH 80']
SPEC_CODES = ['ASME_B31.3','NACE_MR0175']
TEST_TEMP = -196

bom = []
for i in range(1, NUM_LINES+1):
    mat = np.random.choice(MATERIAL_CODES, p=[0.6,0.4])
    bom.append({
        'bom_line_id': i,
        'material_code': mat,
        'nps': np.random.choice(NPS_OPTIONS, p=[0.1,0.1,0.2,0.2,0.15,0.15,0.1]),
        'schedule': np.random.choice(SCHEDULE_OPTIONS, p=[0.7,0.3]),
        'spec_code': 'ASME_B31.3' if mat=='304L_SS' else np.random.choice(SPEC_CODES),
        'quantity': np.random.randint(1,21),
        'test_temp_c': TEST_TEMP
    })
bom_df = pd.DataFrame(bom)

# === 3. Supplier-Material Links ===
links = []
for _, row in supplier_master.iterrows():
    links.append({'supplier_id': row['supplier_id'], 'material_code':'304L_SS'})
    if np.random.rand()<0.6:
        links.append({'supplier_id': row['supplier_id'], 'material_code':'9%_Ni_Alloy'})
sup_mat_df = pd.DataFrame(links)

# === 4. Historical POs (past year) ===
HIST_START = date(2024,5,10)
HIST_END = date(2025,5,9)
HIST_DAYS = (HIST_END-HIST_START).days

po_hist_h = []
po_hist_i = []
ship_hist = []
insp_hist = []

for _, sup in supplier_master.iterrows():
    n = np.random.randint(12,25)
    for idx in range(1,n+1):
        pd_h = HIST_START + timedelta(days=np.random.randint(0,HIST_DAYS))
        po_id = f"HPO{sup['supplier_id']}_{idx}"
        po_hist_h.append({
            'po_number':po_id,'supplier_id':sup['supplier_id'],
            'po_date':pd_h,'vendor_ref':faker.bothify('VR-####'),
            'currency_code':CURRENCY_CODE,'payment_terms':sup['payment_terms']
        })
        line_count = np.random.randint(3,8)
        max_del = pd_h
        for ln in range(1,line_count+1):
            mat = np.random.choice(['304L_SS','9%_Ni_Alloy','VALVE','FLANGE','GASKET','INSULATION'],
                                   p=[0.2,0.2,0.2,0.2,0.1,0.1])
            qty = np.random.randint(1,21)
            lead = np.random.randint(25,50) if 'Ni' in mat or 'SS' in mat else np.random.randint(10,30)
            deliv = pd_h + timedelta(days=lead)
            unit = round(np.random.uniform(80,400),2)
            po_hist_i.append({
                'po_number':po_id,'line_number':ln,'material_id':mat,
                'quantity_order':qty,'uom':'ft' if any(n in mat for n in ['SS','Ni']) else 'pcs',
                'delivery_date':deliv,'unit_price':unit,'line_total':round(unit*qty,2)
            })
            max_del = max(max_del,deliv)
        for s in range(np.random.choice([1,2],p=[0.7,0.3])):
            ship_d = pd_h + timedelta(days=np.random.randint(1,(max_del-pd_h).days))
            arr_min = ship_d + timedelta(days=1)
            arr_max = max_del + timedelta(days=5)
            arr_d = arr_min + timedelta(days=np.random.randint(0,(arr_max-arr_min).days+1))
            status = 'Delivered' if arr_d<=max_del else 'Delayed'
            sid = f"{po_id}-S{s+1}"
            ship_hist.append({
                'shipment_id':sid,'po_number':po_id,
                'shipped_date':ship_d,'arrival_date':arr_d,
                'carrier':faker.company(),'tracking_no':faker.bothify('TRK-#####'),
                'status':status
            })
            insp_hist.append({
                'inspection_id':f"{sid}-I",'shipment_id':sid,
                'inspection_date':arr_d+timedelta(days=np.random.randint(1,5)),
                'pass_fail':np.random.rand()<0.92,
                'notes':'OK' if np.random.rand()<0.92 else faker.sentence(nb_words=6)
            })

# === 5. Bayou Freeze POs (use same batching logic) ===
# Grouping by supplier and spec, fixed batch size 15
po_bay_h = []
po_bay_i = []
for sup_id, group in bom_df.assign(supplier_id=bom_df['material_code'].apply(lambda m: np.random.choice(sup_mat_df[sup_mat_df['material_code']==m]['supplier_id']))).groupby('supplier_id'):
    lines = group.to_dict('records')
    for batch in range(0,len(lines),15):
        batch_lines = lines[batch:batch+15]
        po_id = f"PO{10000+sup_id}_{batch//15+1}"
        pd_b = TODAY + timedelta(days=np.random.randint(0,30))
        po_bay_h.append({
            'po_number':po_id,'supplier_id':sup_id,
            'po_date':pd_b,'vendor_ref':faker.bothify('VR-####'),
            'currency_code':CURRENCY_CODE,'payment_terms':supplier_master.loc[supplier_master['supplier_id']==sup_id,'payment_terms'].iloc[0]
        })
        max_del = pd_b
        for ln in batch_lines:
            lead = 25 if ln['spec_code']=='ASME_B31.3' else 35
            deliv = pd_b + timedelta(days=np.random.randint(lead,lead+10))
            unit = round(np.random.uniform(100,500),2)
            po_bay_i.append({
                'po_number':po_id,'line_number':int(ln['bom_line_id']),
                'material_id':ln['material_code'],'quantity_order':int(ln['quantity']),
                'uom':'ft' if ln['nps'] in NPS_OPTIONS else 'pcs',
                'delivery_date':deliv,'unit_price':unit,'line_total':round(unit*ln['quantity'],2)
            })
            max_del = max(max_del,deliv)

# === Export All Tables ===
tables = {
    'supplier_master.csv': supplier_master,
    'bom.csv': bom_df,
    'supplier_materials.csv': sup_mat_df,
    'po_header_historical.csv': pd.DataFrame(po_hist_h),
    'po_item_historical.csv': pd.DataFrame(po_hist_i),
    'shipments_historical.csv': pd.DataFrame(ship_hist),
    'inspections_historical.csv': pd.DataFrame(insp_hist),
    'po_header_bayou.csv': pd.DataFrame(po_bay_h),
    'po_item_bayou.csv': pd.DataFrame(po_bay_i)
}
for filename, df in tables.items():
    df.to_csv(filename, index=False)


notes - 
inspection notes hilariously awful

In [ ]:
# import pandas as pd
# import numpy as np

# # Load consolidated data
# po_items_hist = pd.read_csv('po_item_historical.csv', parse_dates=['delivery_date'])
# po_items_bay = pd.read_csv('po_item_bayou.csv', parse_dates=['delivery_date'])
# po_items = pd.concat([po_items_hist, po_items_bay], ignore_index=True)

# shipments = pd.read_csv('shipments_historical.csv', parse_dates=['arrival_date'])
# inspections = pd.read_csv('inspections_historical.csv', parse_dates=['inspection_date'])

# # Merge shipments with PO items on po_number to get delivery_date
# df = shipments.merge(po_items, on='po_number', how='left')

# # Calculate lateness
# df['days_late'] = (df['arrival_date'] - df['delivery_date']).dt.days

# # Merge inspection pass/fail
# df = df.merge(inspections[['shipment_id', 'pass_fail']], on='shipment_id', how='left')

# # Define on_time: arrival_date <= delivery_date AND inspection passed
# df['on_time'] = (df['days_late'] <= 0) & (df['pass_fail'] == True)

# # Compute OTIF per supplier
# # First get supplier per shipment via po_items mapping
# po_headers_hist = pd.read_csv('po_header_historical.csv')
# po_headers_bay = pd.read_csv('po_header_bayou.csv')
# po_headers = pd.concat([po_headers_hist, po_headers_bay], ignore_index=True)
# # Merge supplier_id into df
# df = df.merge(po_headers[['po_number', 'supplier_id']], on='po_number', how='left')

# perf = (
#     df.groupby('supplier_id')
#       .agg(total_shipments=('shipment_id','nunique'),
#            on_time_shipments=('on_time','sum'))
#       .reset_index()
# )
# perf['otif_percent'] = np.round(100 * perf['on_time_shipments'] / perf['total_shipments'], 1)
# perf['defect_rate'] = np.round(100 * (1 - perf['on_time_shipments'] / perf['total_shipments']), 1)

# # Export
# perf.to_csv('supplier_performance.csv', index=False)




In [5]:
import pandas as pd
import numpy as np

# 1. Load your tables (adjust paths as needed)
po_items = pd.read_csv('po_item_historical.csv', parse_dates=['delivery_date'])
shipments = pd.read_csv('shipments_historical.csv', parse_dates=['arrival_date'])
inspections = pd.read_csv('inspections_historical.csv', parse_dates=['inspection_date'])
po_headers = pd.read_csv('po_header_historical.csv')

# 2. Merge shipments with PO promise dates & supplier
df_ship = (
    shipments
    .merge(po_items[['po_number','delivery_date']], on='po_number', how='left')
    .merge(po_headers[['po_number','supplier_id']], on='po_number', how='left')
)
df_ship['days_late'] = (df_ship['arrival_date'] - df_ship['delivery_date']).dt.days
df_ship['on_time']  = df_ship['days_late'] <= 0

# 3. Compute OTIF per supplier
otif = (
    df_ship.groupby('supplier_id')
           .agg(total_shipments=('shipment_id','count'),
                on_time_shipments=('on_time','sum'))
           .reset_index()
)
otif['otif_percent'] = (otif['on_time_shipments'] / otif['total_shipments'] * 100).round(1)

# 4. Compute Defect Rate per supplier from inspections
df_insp = (
    inspections
    .merge(shipments[['shipment_id','po_number']], on='shipment_id', how='left')
    .merge(po_headers[['po_number','supplier_id']], on='po_number', how='left')
)
defect = (
    df_insp.groupby('supplier_id')
           .agg(total_inspections=('inspection_id','count'),
                failed_inspections=('pass_fail', lambda x: (~x).sum()))
           .reset_index()
)
defect['defect_rate'] = (defect['failed_inspections'] / defect['total_inspections'] * 100).round(1)

# 5. Combine into a single performance table
perf = otif.merge(defect[['supplier_id','defect_rate']], on='supplier_id')

# Export
perf.to_csv('supplier_performance.csv', index=False)
perf.head()


,supplier_id,total_shipments,on_time_shipments,otif_percent,defect_rate
0,1,84,17,20.2,6.2
1,2,75,23,30.7,21.4
2,3,111,50,45.0,9.1
3,4,96,29,30.2,5.0
4,5,75,27,36.0,0.0


In [7]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

# Configuration
faker = Faker()
np.random.seed(42)
TODAY = datetime(2025, 5, 10)

# Load existing data
suppliers = pd.read_csv('supplier_master.csv')
sup_mat = pd.read_csv('supplier_materials.csv')

# 1. PAYMENT_TERMS
payment_terms = pd.DataFrame([
    {"term_code": "NET_30", "description": "Payment due 30 days after invoice date"},
    {"term_code": "NET_45", "description": "Payment due 45 days after invoice date"}
])
payment_terms.to_csv('payment_terms.csv', index=False)

# 2. CURRENCY
currency = pd.DataFrame([
    {"currency_code": "USD", "currency_name": "U.S. Dollar", "decimal_places": 2}
])
currency.to_csv('currency.csv', index=False)

# 3. MATERIALS
material_list = [
    ("304L_SS", "Stainless Steel 304L Pipe"),
    ("9%_Ni_Alloy", "9% Nickel Alloy Pipe"),
    ("VALVE", "Cryogenic Control Valve"),
    ("FLANGE", "Pipe Flange"),
    ("GASKET", "Industrial Gasket"),
    ("INSULATION", "Cryogenic Pipe Insulation")
]
materials = pd.DataFrame([{"material_id": i+1, "material_code": code, "description": desc} 
                          for i, (code, desc) in enumerate(material_list)])
materials.to_csv('materials.csv', index=False)

# 4. WELD_PROCEDURE_SPECS
wps_records = []
for idx, sup in suppliers.iterrows():
    # 20% chance of certification
    certified = np.random.rand() < 0.2
    assessed_at = TODAY - timedelta(days=np.random.randint(0, 365))
    wps_records.append({
        "wps_id": idx + 1,
        "supplier_id": sup["supplier_id"],
        "impact_test_temp_c": -196,
        "certified": certified,
        "spec_doc_ref": f"wps_cert_{sup['supplier_id']}.pdf" if certified else "",
        "assessed_at": assessed_at.date()
    })
weld_specs = pd.DataFrame(wps_records)
weld_specs.to_csv('weld_procedure_specs.csv', index=False)

# 5. SUPPLIER_CONTRACTS
contract_records = []
contract_id = 1
for _, link in sup_mat.iterrows():
    sup_id = link["supplier_id"]
    mat_code = link["material_code"]
    # Find material_id
    mat_id = materials.loc[materials["material_code"] == mat_code, "material_id"].values[0]
    # Price base
    unit_price = round(np.random.uniform(100, 500), 2)
    # Premium if certified
    is_cert = weld_specs.loc[weld_specs["supplier_id"] == sup_id, "certified"].values[0]
    premium = round(np.random.uniform(0.1, 0.2) * 100, 1) if is_cert else 0.0
    valid_from = TODAY - timedelta(days=30)
    valid_to = TODAY + timedelta(days=365)
    contract_records.append({
        "contract_id": contract_id,
        "supplier_id": sup_id,
        "material_id": mat_id,
        "valid_from": valid_from.date(),
        "valid_to": valid_to.date(),
        "file_ref": f"contract_{contract_id}.pdf",
        "price_premium_pct": premium,
        "unit_price": unit_price,
        "currency_code": "USD"
    })
    contract_id += 1
supplier_contracts = pd.DataFrame(contract_records)
supplier_contracts.to_csv('supplier_contracts.csv', index=False)

# 6. SUPPLIER_DOCUMENTS
doc_records = []
doc_id = 1
subtypes = ["RFQ_Archive", "Contract", "Cert"]
for _, sup in suppliers.iterrows():
    # RFQ archive doc
    doc_records.append({
        "doc_id": doc_id,
        "supplier_id": sup["supplier_id"],
        "doc_type": "RFQ",
        "doc_subtype": "RFQ_Archive",
        "file_ref": f"rfq_archive_{sup['supplier_id']}.docx",
        "uploaded_at": TODAY.date()
    })
    doc_id += 1
    # Certification doc if certified
    if weld_specs.loc[weld_specs["supplier_id"] == sup["supplier_id"], "certified"].values[0]:
        doc_records.append({
            "doc_id": doc_id,
            "supplier_id": sup["supplier_id"],
            "doc_type": "WPS_Cert",
            "doc_subtype": "Cert",
            "file_ref": f"wps_cert_{sup['supplier_id']}.pdf",
            "uploaded_at": weld_specs.loc[weld_specs["supplier_id"] == sup["supplier_id"], "assessed_at"].values[0]
        })
        doc_id += 1

supplier_documents = pd.DataFrame(doc_records)
supplier_documents.to_csv('supplier_documents.csv', index=False)



note - need to generate historical rfq's
then change some stuff here
